In [3]:
import numpy as np
import pandas as pd


In [25]:
class Xl_Regression:
    def __init__(self, iterations=10001):
#         self.params_w = np.ones(len(data_x[0])) * 2
#         self.param_b = 1
        self.learning_rate = 1.0
#         self.lr_b = 0
#         self.lr_w = np.zeros(len(self.params_w))
        iterations = iterations
        self.fit = False
    
    def param_grad(self, x_train_data, y_train_data, params_w, param_b):
        w_grad = np.zeros(len(params_w))
        b_grad = 0.0
        for index_y in range(len(y_train_data)):
            total_y_cal = 0.0
            for index_w in range(len(params_w)):
                total_y_cal += param_b + params_w[index_w] * x_train_data[index_y][index_w]
            # 计算每个w的梯度
            for index_w in range(len(params_w)):
                w_grad[index_w] -= 2 * (y_train_data[index_y] - total_y_cal) * x_train_data[index_y][index_w]
            # 计算b的梯度
            b_grad -= 2 * (y_train_data[index_y] - total_y_cal)
        return (w_grad, b_grad)
        
    def fit(self, X_train, y_train):
        self.fit = True
        self.params_w = np.ones(len(X_train[0])) * 2
        self.param_b = 1
        lr_b = 0
        lr_w = np.zeros(len(self.params_w))
        for i in range(self.iterations):
            w_grad, b_grad = self.param_grad(X_train, y_train, self.params_w, self.param_b)
            lr_b = lr_b + b_grad ** 2
            for index_w in range(len(w_grad)):
                lr_w[index_w] = lr_w[index_w] + w_grad[index_w] ** 2
                self.params_w[index_w] -= learning_rate / np.sqrt(lr_w[index_w]) * w_grad[index_w]
            self.param_b -= learning_rate / np.sqrt(lr_b) * b_grad
            # get loss
            if i % 50 == 0:
                loss = 0.0
                for index_y in range(len(y_train_data)):
                    total_y_cal = 0.0
                    for index_w in range(len(params_w)):
                        total_y_cal += param_b + params_w[index_w] * x_train_data[index_y][index_w]
                    loss += (y_train_data[index_y] - total_y_cal) ** 2
                print("index: %d, loss: %f" % (i, loss))
        
    def predict(self, X_test):
        return X_test * self.params_w + self.param_b
    

In [ ]:
# 数据处理
# 读取数据
df_train = pd.read_csv('./ml2020spring-hw1/train.csv',encoding ='big5')
# 按每天的数据进行分割
df_subs = []
split_size = 18
days = 4320 / split_size
for index in range(int(days)):
    begin = split_size * index
    end = begin + split_size
    df_subs.append(df.iloc[begin: end])
# 合并为新的格式
df_append = pd.DataFrame()
for df_day in df_subs:
    df_day.set_index('測項', inplace=True, drop=False)
    df_append = df_append.append(df_day.iloc[:, 3:-1].stack().unstack(0))

column_list = ['AMB_TEMP', 'CH4', 'CO', 'NMHC', 'NO', 'NO2', 
                                  'NOx', 'O3', 'PM10', 'PM2.5', 'RAINFALL', 'RH', 
                                  'SO2', 'THC', 'WD_HR', 'WIND_DIREC', 'WIND_SPEED', 'WS_HR']
# 将object转为float
for col in column_list:
    if col != 'RAINFALL':
        print(col)
        df_append[col] = df_append[col].astype('float64')
# 将'NR'转为0
df_append.loc[df_append["RAINFALL"] =='NR', 'RAINFALL'] = 0
df_append['RAINFALL'] = df_append['RAINFALL'].astype('float64')
# 归一化
df_to_one = (df_append- df_append.min()) / (df_append.max() - df_append.min())

# 处理test数据集
df_test = pd.read_csv('./ml2020spring-hw1/test.csv',encoding ='big5', header=None)
df_test = df_test.loc[df_test[1] !='RAINFALL'] # 去掉'RAINFALL'
for i in range(2,11):
    df_test[i] = df_test[i].astype('float64')
column_list = ['NO', 'NO2', 'NOx', 'O3', 'PM2.5']
test_x_list = []
for col in column_list:
    df_col_test_x = df_test.loc[df_test[1] ==col, 2:]
    # test归一化
    df_col_test_x = (df_col_test_x - df_append.min()[col]) / (df_append.max()[col] - df_append.min()[col])
    test_x_list.append(df_col_test_x.to_numpy())


In [40]:
# 与PM2.5相关的几个项, 通过前9个数据预测第10个数据的模型
column_list = ['NO', 'NO2', 'NOx', 'O3', 'PM2.5']
test_list = []
model_list = []
for col in column_list:
    df_col_list = []
    for i in range(10):
        df_tmp = df_to_one[col].iloc[i:i - 10,]
        df_tmp.reset_index(drop=True, inplace=True)
        df_col_list.append(df_tmp)
    df_col = pd.concat(df_col_list, axis=1)
    rg_col = Xl_Regression()
    rg_col.fit(df_col[range(9)].to_numpy(), df_col[9].to_numpy())
    model_list.append(rg_col)

# 训练通过其他项的数据预测PM2.5的数据的模型
column_list = ['NO', 'NO2', 'NOx', 'O3']
model_PM = Xl_Regression()
model.fit(df_to_one[column_list].to_numpy(), df_to_one['PM2.5'].to_numpy())


0     id_99
1     WS_HR
2       358
3       357
4       358
5       357
6       354
7       359
8       358
9       359
10      356
dtype: object

In [ ]:
# 预测
test_y_list = []
for index, model in enumerate(model_list):
    test_y_list.append(model.predict(test_x_list[index]))
test_x_pm = array(test_y_list[0:-1]).T
test_y_pm = model_PM.predict(test_x_pm)
test_y_pm = (test_y_pm + test_y_list[-1])/2
test_y_pm = (df_append.max()['PM2.5'] - df_append.min()['PM2.5']) * test_y_pm + df_append.min()['PM2.5']

    

In [ ]:
df_result = pd.read_csv('./ml2020spring-hw1/sample_submission.csv',encoding ='big5')
df_result['value'] = test_y_pm
df_result.to_csv('./result.csv', sep=',', header=True, index=False)